# Collect Satellite images of the farmers location from multiple timestamps

In [ ]:
from python_scripts.connector import *
from python_scripts.helper import *

from eval_scripts.evalscript_true_color import *
from eval_scripts.evalscript_ndvi import *
from eval_scripts.evalscript_ndvi_values  import *

from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient,DataCollection, bbox_to_dimensions, SHConfig
from sentinelhub.geo_utils import to_wgs84
from sentinelhub.constants import CRS
from sentinelhub import SentinelHubCatalog

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


## Step 1: Get the necessary timestamps

In [ ]:
def get_time_slots(start, end, chunks):
    """
    Get the avalable time windows (time slots)
    """ 
    datetime_start = datetime.datetime.strptime(start, '%Y-%m-%d')
    datetime_end   = datetime.datetime.strptime(end, '%Y-%m-%d')
    
    tdelta = (datetime_end - datetime_start) / chunks
    edges = [(datetime_start + i*tdelta).date().isoformat() for i in range(chunks)]
    slots = [(edges[i], edges[i+1]) for i in range(len(edges)-1)]
    return slots
    

In [ ]:
slots = get_time_slots('2017-01-01', '2022-01-01', 5)
print("Yearly time windows")
for slot in slots:
    print(slot)

## Step 2: Get the necessary coordinates 

In [ ]:
# convert coordinations_file (csv file) into a dataframe
coordinations_df = pd.read_excel('data/locations/FarmerLocationExtract4Interns_sentinel-hub.xlsx')

In [ ]:
# Based on how far wail is with the conversion of the coordinates of the farmers location, we need to check whether or 
# the coordination needs to mdofied. 
coordinations_df

In [ ]:
coordinations_df.columns

In [ ]:
def bbox_converter(x):
    """
    Convert coordinates (longitude, latitude) in to a bbox in WGS84 format.
    
    return [long, lat, long, lat]
    """
    lng, lat = to_wgs84(x['P1_Longitude'], x['P1_Latitude'], CRS.WGS84)
    coords = [lng - 0.1, lat - 0.1, lng + 0.1, lat + 0.1]
    return coords

In [ ]:
geolocations  = coordinations_df[['P1_Longitude', 'P1_Latitude']].copy()

In [ ]:
geolocations[geolocations['P1_Latitude'].str.contains('"')]

In [ ]:
geolocations['P1_Latitude'] = geolocations['P1_Latitude'].str.replace('"', '')

In [ ]:
geolocations[geolocations['P1_Latitude'].str.contains('"')]

In [ ]:
geolocations = geolocations.astype("float64")

In [ ]:
geolocations.dtypes

## Create requests for the labels

In [ ]:
def map_ndvi_label(ndvi):
    """
    Categorizing range of ndvi values to a label.
    We have two categories: 
    1. trees - 1
    2. not trees - 0
    We categorize evey ndvi values that is higher than our threshold(0.6) to be trees (1).
    Everything that is equal or less than our threshold (0.6) we be 'not trees' (0): 
    """
    min_ndvi = 0.6
    ndvi_copy = ndvi.copy()

    labels = np.where(ndvi_copy > min_ndvi, 1, 0)
     
    return labels

In [ ]:
def labels_request(coords, slot, config):
    """
    Download labels in binary array from sentinel hub.
    
    return: numpy array of 0 and 1
    """
    ndvi_values = sentinel_request(evalscript_ndvi_values, coords, slot, config, False, other_args=least_clouds())
    ndvi_labels= map_ndvi_label(ndvi_values)
    
    return ndvi_labels
    

## Download all the satellite images 

In [ ]:
import time # time the performance

In [ ]:
def least_clouds():
    return { 
    "dataFilter": { 
        "maxCloudCoverage": 10
        } 
    } 

In [ ]:
for index, row in geolocations.iterrows(): # iterate through each location
    
    coords = bbox_converter(row) # convert each location to a bbox
    
    requests = []
    labels = [] # TODO check if this is the correct way to store the labels 
    
    start = time.perf_counter()
    for slot in slots:
        img_true = sentinel_request(evalscript_true_color, coords,slot, config, True, other_args=least_clouds())
        img_ndvi = sentinel_request(evalscript_ndvi      , coords,slot, config, True, other_args=least_clouds())
        requests.extend([img_true, img_ndvi])
        
        # get the labels
        binary_arr_labels = labels_request(coords, slot, config)
        labels.append(binary_arr_labels) # TODO  check if this is correct
    end = time.perf_counter()
    print("Done in: {} seconds".format( end - start))

Below you wil find a function the plot images for one specific scene/area and the given timestamps

In [ ]:
def plot_images(data, slots):
    # some stuff for pretty plots
    ncols = 2
    nrows = 2
    aspect_ratio = betsiboka_size[0] / betsiboka_size[1]
    subplot_kw = {'xticks': [], 'yticks': [], 'frame_on': False}

    fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(5 * ncols * aspect_ratio, 5 * nrows),
                            subplot_kw=subplot_kw)

    for idx, image in enumerate(data):
        ax = axs[idx // ncols][idx % ncols]
        ax.imshow(np.clip(image * 2.5/255, 0, 1))
        ax.set_title(f'{slots[idx][0]}  -  {slots[idx][1]}', fontsize=10)

    plt.tight_layout()